In [34]:
from __future__ import division, print_function, absolute_import
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import numpy
import PIL
from PIL import Image
np.random.seed(1337)  # for reproducibility

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [35]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10

# Network Parameters
num_hidden_1 = 64 # 1st layer num features
num_hidden_2 = 16 # 2nd layer num features (the latent dim)
num_input = 43 
num_classes = 5

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_2, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [36]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [37]:
from numpy import genfromtxt
my_data = genfromtxt('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/Cgamma_VVc.csv', delimiter=',')

In [38]:
data = my_data[1:,2:]
print(data.shape)

(291, 43)


In [39]:
print(data.max())

0.746019959449768


In [40]:
data = np.array(data)
print(data.shape)

(291, 43)


In [41]:
import pandas
colnames = ['Latitude', 'Longitude', 'CType', 'Remarks', 'Index']
data1 = pandas.read_csv('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/CropClean.csv', names=colnames)

In [42]:
ctype = data1.CType.tolist()
print(ctype)

['CType', 'PD', 'PD', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'SC', 'SC', 'SC', 'JW', 'PD', 'SC', 'BA', 'SC', 'BA', 'PD', 'PD', 'SC', 'PP', 'PD', 'PD', 'PD', 'PD', 'PD', 'BA', 'SC', 'SC', 'BA', 'BA', 'JW', 'JW', 'SC', 'PD', 'BA', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'BA', 'SC', 'PD', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'PD', 'PD', 'SC', 'JW', 'PD', 'PD', 'BA', 'BA', 'SC', 'SC', 'SC', 'BA', 'PP', 'PP', 'BA', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'PD', 'PD', 'SC', 'PD', 'SC', 'PD', 'BA', 'JW', 'PD', 'SC', 'JW', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'SC', 'SC', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'JW', 'PD', 'SC', 'SC', 'JW', 

In [43]:
p = np.array(ctype)
print(p.shape)

(292,)


In [44]:
lbl1 = ctype
label = []
for i in range(1,292):
    if(lbl1[i] == "PD"):
        label += [0]
    if(lbl1[i] == "JW"):
        label += [1]
    if(lbl1[i] == "PP"):
        label += [2]
    if(lbl1[i] == "SC"):
        label += [3]
    if(lbl1[i] == "BA"):
        label += [4]
print(label)

[0, 0, 0, 3, 1, 3, 3, 3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 1, 0, 3, 4, 3, 4, 0, 0, 3, 2, 0, 0, 0, 0, 0, 4, 3, 3, 4, 4, 1, 1, 3, 0, 4, 1, 3, 3, 3, 3, 3, 0, 0, 4, 3, 0, 0, 3, 3, 3, 3, 3, 1, 0, 0, 3, 1, 0, 0, 4, 4, 3, 3, 3, 4, 2, 2, 4, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 1, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 0, 0, 3, 0, 3, 0, 4, 1, 0, 3, 1, 3, 3, 1, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 3, 1, 3, 3, 3, 3, 3, 3, 0, 1, 0, 3, 3, 1, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 0, 0, 0, 0, 1, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 4, 4, 3, 0, 4, 0, 3, 3, 3, 3, 0, 3, 0, 3, 2, 3, 0, 3, 1, 4, 3, 0, 0, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 4, 1, 0, 0, 2, 3, 3, 3, 0, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 0, 4, 4, 4, 0, 0, 0, 0, 3, 3, 3, 3, 4, 3, 3, 4, 4, 3, 3, 1, 3, 4, 1, 3, 3, 3, 3, 4, 0, 0]


In [45]:
label = np.array(label)
print(label.shape)

(291,)


In [46]:
# 80% train 20% test (NDVIclean)
X_train = data[0:232,:]
X_test = data[232:,:]
y_train = label[0:232]
y_test = label[232:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(232, 43)
(59, 43)
(232,)
(59,)


In [47]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(232, 5)
(59, 5)


In [48]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 13

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.95478604
Epoch: 2 cost = 0.87323350
Epoch: 3 cost = 0.80715521
Epoch: 4 cost = 0.75550604
Epoch: 5 cost = 0.71587210
Epoch: 6 cost = 0.68533657
Epoch: 7 cost = 0.66144852
Epoch: 8 cost = 0.64238779
Epoch: 9 cost = 0.62669593
Epoch: 10 cost = 0.61315676
Epoch: 11 cost = 0.60085042
Epoch: 12 cost = 0.58918201
Epoch: 13 cost = 0.57782415
Epoch: 14 cost = 0.56662130
Epoch: 15 cost = 0.55550531
Epoch: 16 cost = 0.54444009
Epoch: 17 cost = 0.53339168
Epoch: 18 cost = 0.52231567
Epoch: 19 cost = 0.51115368
Epoch: 20 cost = 0.49983441
Epoch: 21 cost = 0.48828195
Epoch: 22 cost = 0.47643944
Epoch: 23 cost = 0.46431661
Epoch: 24 cost = 0.45204786
Epoch: 25 cost = 0.43989746
Epoch: 26 cost = 0.42819905
Epoch: 27 cost = 0.41734552
Epoch: 28 cost = 0.40780220
Epoch: 29 cost = 0.39996360
Epoch: 30 cost = 0.39392589
Epoch: 31 cost = 0.38942365
Epoch: 32 cost = 0.38599132
Epoch: 33 cost = 0.38317603
Epoch: 34 cost = 0.38064870
Epoch: 35 cost = 0.37821194
Epoch: 36 cost = 0.37576365
E

Epoch: 308 cost = 0.00203368
Epoch: 309 cost = 0.00214153
Epoch: 310 cost = 0.00280836
Epoch: 311 cost = 0.03140652
Epoch: 312 cost = 0.01958491
Epoch: 313 cost = 0.00214875
Epoch: 314 cost = 0.00165955
Epoch: 315 cost = 0.00153363
Epoch: 316 cost = 0.00146317
Epoch: 317 cost = 0.00142709
Epoch: 318 cost = 0.00141888
Epoch: 319 cost = 0.00140409
Epoch: 320 cost = 0.00138653
Epoch: 321 cost = 0.00138559
Epoch: 322 cost = 0.00139303
Epoch: 323 cost = 0.00137968
Epoch: 324 cost = 0.00137191
Epoch: 325 cost = 0.00139540
Epoch: 326 cost = 0.00139838
Epoch: 327 cost = 0.00137775
Epoch: 328 cost = 0.00135821
Epoch: 329 cost = 0.00139804
Epoch: 330 cost = 0.00138860
Epoch: 331 cost = 0.00142993
Epoch: 332 cost = 0.00143834
Epoch: 333 cost = 0.00151196
Epoch: 334 cost = 0.00167745
Epoch: 335 cost = 0.00208631
Epoch: 336 cost = 0.00508366
Epoch: 337 cost = 0.03145061
Epoch: 338 cost = 0.00239433
Epoch: 339 cost = 0.00147032
Epoch: 340 cost = 0.00121737
Epoch: 341 cost = 0.00111463
Epoch: 342 cos

Epoch: 727 cost = 0.00016050
Epoch: 728 cost = 0.00021342
Epoch: 729 cost = 0.00020209
Epoch: 730 cost = 0.00020222
Epoch: 731 cost = 0.00020056
Epoch: 732 cost = 0.00017011
Epoch: 733 cost = 0.00013101
Epoch: 734 cost = 0.00013640
Epoch: 735 cost = 0.00020203
Epoch: 736 cost = 0.00022624
Epoch: 737 cost = 0.00017264
Epoch: 738 cost = 0.00014670
Epoch: 739 cost = 0.00020068
Epoch: 740 cost = 0.00018695
Epoch: 741 cost = 0.00018373
Epoch: 742 cost = 0.00017508
Epoch: 743 cost = 0.00014802
Epoch: 744 cost = 0.00016566
Epoch: 745 cost = 0.00015998
Epoch: 746 cost = 0.00013595
Epoch: 747 cost = 0.00018278
Epoch: 748 cost = 0.00019008
Epoch: 749 cost = 0.00017415
Epoch: 750 cost = 0.00021720
Epoch: 751 cost = 0.00016025
Epoch: 752 cost = 0.00017407
Epoch: 753 cost = 0.00021141
Epoch: 754 cost = 0.00017350
Epoch: 755 cost = 0.00016634
Epoch: 756 cost = 0.00019389
Epoch: 757 cost = 0.00017614
Epoch: 758 cost = 0.00016348
Epoch: 759 cost = 0.00015662
Epoch: 760 cost = 0.00020092
Epoch: 761 cos

In [49]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [21]:
print(y_test.shape)

(59, 5)


In [50]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.66101694


In [28]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data[0:204,:]
X_test = data[204:,:]
y_train = label[0:204]
y_test = label[204:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(204, 43)
(87, 43)
(204,)
(87,)


In [29]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(204, 5)
(87, 5)


In [32]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 1.36526863
Epoch: 2 cost = 1.18761202
Epoch: 3 cost = 1.03394393
Epoch: 4 cost = 0.90846682
Epoch: 5 cost = 0.81204665
Epoch: 6 cost = 0.74157834
Epoch: 7 cost = 0.69105494
Epoch: 8 cost = 0.65367947
Epoch: 9 cost = 0.62371097
Epoch: 10 cost = 0.59719402
Epoch: 11 cost = 0.57181764
Epoch: 12 cost = 0.54650187
Epoch: 13 cost = 0.52110740
Epoch: 14 cost = 0.49641854
Epoch: 15 cost = 0.47418497
Epoch: 16 cost = 0.45646699
Epoch: 17 cost = 0.44411105
Epoch: 18 cost = 0.43614020
Epoch: 19 cost = 0.43087743
Epoch: 20 cost = 0.42704717
Epoch: 21 cost = 0.42396337
Epoch: 22 cost = 0.42130905
Epoch: 23 cost = 0.41893494
Epoch: 24 cost = 0.41675554
Epoch: 25 cost = 0.41470975
Epoch: 26 cost = 0.41274822
Epoch: 27 cost = 0.41082974
Epoch: 28 cost = 0.40891963
Epoch: 29 cost = 0.40698822
Epoch: 30 cost = 0.40501006
Epoch: 31 cost = 0.40296253
Epoch: 32 cost = 0.40082538
Epoch: 33 cost = 0.39857976
Epoch: 34 cost = 0.39620759
Epoch: 35 cost = 0.39369053
Epoch: 36 cost = 0.39101042
E

Epoch: 299 cost = 0.00364107
Epoch: 300 cost = 0.00364594
Epoch: 301 cost = 0.00364639
Epoch: 302 cost = 0.00365213
Epoch: 303 cost = 0.00364829
Epoch: 304 cost = 0.00360844
Epoch: 305 cost = 0.00352728
Epoch: 306 cost = 0.00341094
Epoch: 307 cost = 0.00328823
Epoch: 308 cost = 0.00321472
Epoch: 309 cost = 0.00315696
Epoch: 310 cost = 0.00307969
Epoch: 311 cost = 0.00300313
Epoch: 312 cost = 0.00294933
Epoch: 313 cost = 0.00286592
Epoch: 314 cost = 0.00272941
Epoch: 315 cost = 0.00261417
Epoch: 316 cost = 0.00249674
Epoch: 317 cost = 0.00236716
Epoch: 318 cost = 0.00230466
Epoch: 319 cost = 0.00231347
Epoch: 320 cost = 0.00240679
Epoch: 321 cost = 0.00440592
Epoch: 322 cost = 0.01612819
Epoch: 323 cost = 0.00391829
Epoch: 324 cost = 0.00170850
Epoch: 325 cost = 0.00154952
Epoch: 326 cost = 0.00149968
Epoch: 327 cost = 0.00145103
Epoch: 328 cost = 0.00142719
Epoch: 329 cost = 0.00142492
Epoch: 330 cost = 0.00142802
Epoch: 331 cost = 0.00143833
Epoch: 332 cost = 0.00144795
Epoch: 333 cos

Epoch: 584 cost = 0.00017446
Epoch: 585 cost = 0.00015737
Epoch: 586 cost = 0.00017009
Epoch: 587 cost = 0.00019622
Epoch: 588 cost = 0.00024546
Epoch: 589 cost = 0.00023241
Epoch: 590 cost = 0.00021261
Epoch: 591 cost = 0.00019625
Epoch: 592 cost = 0.00017257
Epoch: 593 cost = 0.00019375
Epoch: 594 cost = 0.00019034
Epoch: 595 cost = 0.00018885
Epoch: 596 cost = 0.00016434
Epoch: 597 cost = 0.00025357
Epoch: 598 cost = 0.00018877
Epoch: 599 cost = 0.00016895
Epoch: 600 cost = 0.00018094
Epoch: 601 cost = 0.00017405
Epoch: 602 cost = 0.00020768
Epoch: 603 cost = 0.00018131
Epoch: 604 cost = 0.00016375
Epoch: 605 cost = 0.00022083
Epoch: 606 cost = 0.00017064
Epoch: 607 cost = 0.00017585
Epoch: 608 cost = 0.00019543
Epoch: 609 cost = 0.00017861
Epoch: 610 cost = 0.00017756
Epoch: 611 cost = 0.00018324
Epoch: 612 cost = 0.00019422
Epoch: 613 cost = 0.00019610
Epoch: 614 cost = 0.00019305
Epoch: 615 cost = 0.00019571
Epoch: 616 cost = 0.00020332
Epoch: 617 cost = 0.00021586
Epoch: 618 cos

Epoch: 992 cost = 0.00011019
Epoch: 993 cost = 0.00013003
Epoch: 994 cost = 0.00012084
Epoch: 995 cost = 0.00009622
Epoch: 996 cost = 0.00011510
Epoch: 997 cost = 0.00009662
Epoch: 998 cost = 0.00010925
Epoch: 999 cost = 0.00010127
Epoch: 1000 cost = 0.00007191


In [33]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.62068963
